# Analyse a sample of FUAs

This notebook produces face polygons and computes shape metrics for each of them.

In [1]:
# import os
# os.environ['USE_PYGEOS'] = '0'

import warnings

import geopandas
import dask_geopandas
import numpy
import shapely  # needs to be shapely 2
import esda.shape as shape

from tqdm import tqdm

/tmp/ipykernel_3144912/4077717691.py:6: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas


In [2]:
sample = geopandas.read_parquet("../data/sample.parquet")

## Measure shape characteristics

Polygonize the network to get polygons fully enclosed by street network geometry (face polygons) and measure their shape characteristics.

In [3]:
# Loop over unique FUA IDs
for fua_id in tqdm(sample.eFUA_ID, total=len(sample)):
    # Read stret network
    roads = geopandas.read_parquet(f"../data/{int(fua_id)}/roads_osm.parquet")

    # remove service roads
    roads = roads[roads.highway != "service"]

    # Polygonize street network
    polygons = geopandas.GeoSeries(
        shapely.polygonize(  # polygonize
            shapely.node(  # ensure properly nodded non-planar interesections
                shapely.GeometryCollection(  # need a single geometry for nodding
                    roads.geometry.array
                )
            ).geoms # get parts of the collection from nodding
        ).geoms, # get parts of the collection from polygonize
        crs=roads.crs
    ).explode(ignore_index=True) # shoudln't be needed but doesn't hurt to ensure
    
    # Store geometries as a GeoDataFrame
    polygons = geopandas.GeoDataFrame(
        geometry=polygons
    )
    
    # Ensure all polygons are valid. Should not be necessary.
    if not polygons.is_valid.all():
        polygons = geopandas.GeoDataFrame(
        geometry=geopandas.GeoSeries(
            shapely.make_valid(polygons.geometry.array), crs=roads.crs
        ).explode(ignore_index=True)
    )
    
    # Ensure that all geometries are polygons
    if not (polygons.geom_type == "Polygon").all():
        polygons = polygons[polygons.geom_type == "Polygon"].reset_index(drop=True)

    area = polygons.area

    # measure (circular) compactness
    polygons["circular_compactness"] = shape.minimum_bounding_circle_ratio(polygons)
    polygons["circular_compactness_index"] = polygons["circular_compactness"] * area
    
    # isoperimetric_quotient
    polygons["isoperimetric_quotient"] = shape.isoperimetric_quotient(polygons)
    polygons["isoperimetric_quotient_index"] = polygons["isoperimetric_quotient"] * area


    # isoareal_quotient
    polygons["isoareal_quotient"] = shape.isoareal_quotient(polygons)
    polygons["isoareal_quotient_index"] = polygons["isoareal_quotient"] * area

    # radii_ratio
    polygons["radii_ratio"] = shape.radii_ratio(polygons)
    polygons["radii_ratio_index"] = polygons["radii_ratio"] * area

    # diameter_ratio
    polygons["diameter_ratio"] = shape.diameter_ratio(polygons)
    polygons["diameter_ratio_index"] = polygons["diameter_ratio"] * area

    # save polygons to a partitioned GeoParquet
    polygons = dask_geopandas.from_geopandas(polygons, npartitions=10)
    polygons.to_parquet(f"../data/{int(fua_id)}/polygons/")

100%|████████████████████████████████████████| 131/131 [27:14<00:00, 12.48s/it]
